In [88]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json


In [89]:
df = pd.read_csv("Books_rating.csv")

In [90]:
df_book = pd.read_csv("books_data.csv")

In [91]:
#drop_columns = ["description", "image", "previewLink", "publisher", "publishedDate", "infoLink"]

#books_data_cleaned = books_data.drop(drop_columns, axis=1)

#books_data_cleaned

#category_array = df['categories'].unique()

#print('\n'.join(category_array.values))


In [93]:
df

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
...,...,...,...,...,...,...,...,...,...,...
2999995,B000NSLVCU,The Idea of History,NaN,NaN,NaN,14/19,4.0,937612800,Difficult,"This is an extremely difficult book to digest,..."
2999996,B000NSLVCU,The Idea of History,NaN,A1SMUB9ASL5L9Y,jafrank,1/1,4.0,1331683200,Quite good and ahead of its time occasionally,This is pretty interesting. Collingwood seems ...
2999997,B000NSLVCU,The Idea of History,NaN,A2AQMEKZKK5EE4,"L. L. Poulos ""Muslim Mom""",0/0,4.0,1180224000,Easier reads of those not well versed in histo...,"This is a good book but very esoteric. ""What i..."
2999998,B000NSLVCU,The Idea of History,NaN,A18SQGYBKS852K,"Julia A. Klein ""knitting rat""",1/11,5.0,1163030400,"Yes, it is cheaper than the University Bookstore","My daughter, a freshman at Indiana University,..."


In [94]:
df_book

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
212399,The Orphan Of Ellis Island (Time Travel Advent...,"During a school trip to Ellis Island, Dominick...",['Elvira Woodruff'],http://books.google.com/books/content?id=J7M-N...,http://books.google.com/books?id=J7M-NwAACAAJ&...,Scholastic Paperbacks,2000-06-01,http://books.google.com/books?id=J7M-NwAACAAJ&...,['Juvenile Fiction'],2.0
212400,Red Boots for Christmas,Everyone in the village of Friedensdorf is hap...,NaN,http://books.google.com/books/content?id=3n8k6...,http://books.google.com/books?id=3n8k6wl4BbYC&...,NaN,1995,http://books.google.com/books?id=3n8k6wl4BbYC&...,['Juvenile Fiction'],NaN
212401,Mamaw,"Give your Mamaw a useful, beautiful and though...",['Wild Wild Cabbage'],NaN,http://books.google.com/books?id=zytVswEACAAJ&...,NaN,2018-01-17,http://books.google.com/books?id=zytVswEACAAJ&...,NaN,NaN
212402,The Autograph Man,Alex-Li Tandem sells autographs. His business ...,['Zadie Smith'],http://books.google.com/books/content?id=JM6YV...,http://books.google.com/books?id=JM6YVPx_clMC&...,Vintage,2003-08-12,https://play.google.com/store/books/details?id...,['Fiction'],19.0


In [95]:
df.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [96]:
dfhead = df.head()

In [128]:
#Test Preprocessing/Data Cleaning
drop_columns = ["Title", "Price", "profileName", "review/time", "review/summary", "review/helpfulness", "review/text"]

books_data_cleaned = dfhead.drop(drop_columns, axis=1)

books_data_cleaned

,Id,User_id,review/score
0,1882931173,AVCGYZL8FQQTD,4.0
1,0826414346,A30TK6U7DNS82R,5.0
2,0826414346,A3UH4UZ4RSVO82,5.0
3,0826414346,A2MVUWT453QH61,4.0
4,0826414346,A22X4XUPKF66MR,4.0


In [129]:
df_ratings = df

In [130]:
#Preprocessing/Data Cleaning
drop_columns = ["Price", "profileName", "review/time", "review/summary", "review/helpfulness", "review/text"]

books_ratings_cleaned = df_ratings.drop(drop_columns, axis=1)

books_ratings_cleaned

,Id,Title,User_id,review/score
0,1882931173,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,4.0
1,0826414346,Dr. Seuss: American Icon,A30TK6U7DNS82R,5.0
2,0826414346,Dr. Seuss: American Icon,A3UH4UZ4RSVO82,5.0
3,0826414346,Dr. Seuss: American Icon,A2MVUWT453QH61,4.0
4,0826414346,Dr. Seuss: American Icon,A22X4XUPKF66MR,4.0
...,...,...,...,...
2999995,B000NSLVCU,The Idea of History,NaN,4.0
2999996,B000NSLVCU,The Idea of History,A1SMUB9ASL5L9Y,4.0
2999997,B000NSLVCU,The Idea of History,A2AQMEKZKK5EE4,4.0
2999998,B000NSLVCU,The Idea of History,A18SQGYBKS852K,5.0


In [131]:
#Create Dictionary Key for Unique ID:Book
book_dict = dict()
for row in books_ratings_cleaned.itertuples():
    if row[1] not in book_dict:
        book_dict[row[1]] = dict()

    book_dict[row[1]] = row[2]

In [127]:
book_dict

{'1882931173': 'Its Only Art If Its Well Hung!',
 '0826414346': 'Dr. Seuss: American Icon',
 '0829814000': 'Wonderful Worship in Smaller Churches',
 '0595344550': 'Whispers of the Wicked Saints',
 '0253338352': 'Nation Dance: Religion, Identity and Cultural Difference in the Caribbean',
 '0802841899': 'The Church of Christ: A Biblical Ecclesiology for Today',
 'B0007FIF28': 'The Overbury affair (Avon)',
 'B000JINSBG': 'A Walk in the Woods: a Play in Two Acts',
 '0895554224': 'Saint Hyacinth of Poland',
 '0963923080': "Rising Sons and Daughters: Life Among Japan's New Young",
 '0854968350': "Muslim Women's Choices: Religious Belief and Social Reality (Cross Cultural Perspectives on Women)",
 '0918973031': 'Dramatica for Screenwriters',
 '1858683092': 'Mensa Number Puzzles (Mensa Word Games for Kids)',
 '0792391810': 'Vector Quantization and Signal Compression (The Springer International Series in Engineering and Computer Science)',
 'B000879GGE': 'A husband for Kutani',
 '0908968000': '

In [ ]:
drop_columns = ["Title", "Price", "profileName", "review/time", "review/summary", "review/helpfulness", "review/text"]

books_ratings_cleaned = df_ratings.drop(drop_columns, axis=1)

books_ratings_cleaned

In [81]:
books_ratings_cleaned['User_id'].isna()

sum(books_ratings_cleaned['User_id'].isna())

561787

In [82]:
books_ratings_cleaned = books_ratings_cleaned[books_ratings_cleaned['User_id'].notna()]

In [83]:
books_ratings_cleaned

,Id,User_id,review/score
0,1882931173,AVCGYZL8FQQTD,4.0
1,0826414346,A30TK6U7DNS82R,5.0
2,0826414346,A3UH4UZ4RSVO82,5.0
3,0826414346,A2MVUWT453QH61,4.0
4,0826414346,A22X4XUPKF66MR,4.0
...,...,...,...
2999990,B000NSLVCU,AI1QNMVF2E3TN,5.0
2999994,B000NSLVCU,AOFGOUMXLMVZS,4.0
2999996,B000NSLVCU,A1SMUB9ASL5L9Y,4.0
2999997,B000NSLVCU,A2AQMEKZKK5EE4,4.0


In [84]:
#Create Dictionary Keys of the form     'User': {'Book ID': Rating, 'Book ID': Rating,...}
new_data = dict()
for row in books_ratings_cleaned.sample(500).itertuples():
    if row[2] not in new_data:
        new_data[row[2]] = dict()

    new_data[row[2]][row[1]] = row[3]

In [85]:
new_data

{'A3HLJQCY6GKRVK': {'1588209636': 5.0},
 'A19JH6VPUN3A4R': {'B000CBWDUW': 5.0},
 'A605O2Y0JBZGM': {'0743435516': 2.0},
 'A23C4549CCC5P2': {'B000MWDD32': 5.0},
 'APPZQWRWSS3MC': {'B00087H4HW': 5.0},
 'AKKXHJPZQT5YP': {'0873588053': 5.0},
 'A1CGLIDN7E5MK8': {'B000QRBOI4': 5.0},
 'A3W3URDMTJ9I7D': {'B000KXXBXA': 5.0},
 'A10KXAIM5NANWQ': {'B000N76ZCC': 5.0},
 'A163CGRGG0IV5V': {'020135358X': 5.0},
 'AX4L4BSVG8J8Q': {'141591673X': 4.0},
 'AVACTGKPQIZTG': {'B000OTWMY4': 5.0},
 'A3RM1XXZLK464V': {'B0007DSDOU': 1.0},
 'A2GHTSBU7IHIBO': {'078612914X': 5.0},
 'A3PJXZOYC8026Z': {'B000OXCFGK': 3.0},
 'AYZQFY5JK0SNV': {'B000N765OK': 5.0},
 'A15KKNYZK3TMUI': {'0460872702': 5.0},
 'AIHCJYMHZZ7Q9': {'0516053639': 5.0},
 'A6P9HEPEJYM6J': {'B000HMK1SW': 5.0},
 'AC6VWCWB4JEDL': {'B00005X6H7': 5.0, 'B000HHNE0Y': 5.0},
 'A2TMON0H9ZMJXF': {'B0007E1GY8': 3.0},
 'A2SFNACEX04NQT': {'0026276615': 4.0},
 'A1IB2ST60PBAQ1': {'B000I2ZP5K': 5.0},
 'A2IW58O8GNY7FE': {'B000GTETQ6': 4.0},
 'A5TO0ZNZC0JVI': {'1892213060

In [62]:
#convert Dictionary Key into Sparse Matrix With users as Columns, Books as rows
test = pd.DataFrame.from_dict(new_data)

In [63]:
test.head()

,AZ85B5Q1UEH5U,ANPG649N67GQG,AYO8MU18Z8SPV,A1VPWOAXW119PQ,A33LFLWDQ446YN,AWL99NTRKJV88,AVMYCMAOY4F4S,A3G05IEEQP0LHM,A3CAJZ1OFNYELP,AOC59UY1HBMU8,...,A35UIYSKD6PVEG,A1GWDZWFRQXO5X,A1HWKKI923US1R,A3BXK3HE0XUG7,AYVEZG05167UK,A1RBBBHKE3NW3E,A1TRWQJ3FWE2KC,A2AKNCSNUEQKSZ,A14DQTPVOEVK4M,A2ROGO40OHB2MY
B000PJLAJQ,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B000NDSX6C,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
0934770026,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1423319702,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B000N6R2BG,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
#take Matrix Transpose
test2 = test.transpose()

In [65]:
#populate NaN's as zeroes
test2.fillna(0)

,B000PJLAJQ,B000NDSX6C,0934770026,1423319702,B000N6R2BG,B000N6XXN2,B000TNGU5M,B000Q1PXD2,B0006AQ6MA,1582790337,...,B000K76LEW,1594200629,0934125104,0446605174,B000N6QMYE,096421685X,0295984449,0965396657,0613245563,0060923245
AZ85B5Q1UEH5U,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ANPG649N67GQG,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AYO8MU18Z8SPV,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1VPWOAXW119PQ,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A33LFLWDQ446YN,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A1RBBBHKE3NW3E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
A1TRWQJ3FWE2KC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
A2AKNCSNUEQKSZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
A14DQTPVOEVK4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
#high_value_users = [userId for (userId, count) in test.items() if count > 50]